# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Pytorch

---

# Observations et environnement
---

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import importlib
import pandas as pd
import seaborn as sns

## Données

In [3]:
df_text = pd.read_csv('data/tmp/221110-TextsInfox.csv')

In [4]:
df_text

,url,text,infox
0,https://www.breizh-info.com/2022/09/30/208531/...,Réunification. Les six « insoumis » de Bretagn...,0.0
1,https://www.breizh-info.com/2022/10/02/208574/...,"Stonehenge, patrimoine africain ? Ben voyons…\...",0.0
2,https://www.dreuz.info/2022/10/elections-de-mi...,Elections américaines de mi-mandat : suivi quo...,0.0
3,https://www.dreuz.info/2022/10/lukraine-est-en...,L’Ukraine est en train de gagner la guerre\nOn...,0.0
4,https://www.dreuz.info/2022/10/la-fda-va-publi...,La FDA va publier une étude sur les effets ind...,0.0
...,...,...,...
450,https://www.lelibrepenseur.org/sanna-marine-la...,"Sanna Marin, la Première ministre finlandaise,...",0.0
451,https://www.lelibrepenseur.org/manifestations-...,Manifestations importantes d’agriculteurs aux ...,1.0
452,https://www.lelibrepenseur.org/les-deces-exces...,Les décès excessifs sont en augmentation mais ...,1.0
453,https://www.lelibrepenseur.org/1-milliard-de-c...,1 milliard de Chinois touchés par un piratage ...,1.0


In [5]:
with open('data/tmp/221110-IgnoreLemmas.txt','r') as file:
    ignore_lemmas = file.read().split('\n') 

In [6]:
len(ignore_lemmas)

15203

# Experience
---

## Dataloader

In [7]:
df_text.iloc[0].to_dict()

{'url': 'https://www.breizh-info.com/2022/09/30/208531/reunification-les-six-insoumis-de-bretagne/',
 'text': 'Réunification. Les six «\xa0insoumis\xa0» de Bretagne\nDans l’opération que prépare Bretagne réunie, il sera intéressant de suivre le comportement des 37 députés et des 19 sénateurs de Bretagne. A coup sûr, il faudra les assiéger et médiatiser leur attitude pour les contraindre à soutenir l’action de l’association. Un traitement particulier devra être réservé aux six LFI : Murielle Lepvraud (Guingamp), Frédéric Mathieu (Rennes Bruz), Mathilde Hignet (Redon), Andy Kerbrat (Nantes centre), Ségolène Amiot (Nantes-Saint-Herblain), Mathias Tavel (Saint-Nazaire). Leur patron et leur parti étant farouchement jacobins – donc hostiles dans les faits à la cause bretonne -, la logique veut qu’ils adoptent une attitude défavorable au projet de Bretagne réunie.\n\n\nLe « lider maximo » commande et ses ouailles obéissent. « En retrait mais pas en retraite », répète-t-il à l’envi. Mais ces t

In [8]:
import torch

In [9]:
from torch.utils.data import Dataset, DataLoader

In [10]:
class PatatDataset(Dataset):
    """Patat dataset."""

    def __init__(self, df, transform=None):
        self._df = df
        self._transform = transform

    def __len__(self):
        return len(self._df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = self._df.iloc[idx].to_dict()

        if self._transform:
            sample = self._transform(sample)

        return sample

In [11]:
ds = PatatDataset(df_text)

In [12]:
ds[[1,2,3]]

{'url': {1: 'https://www.breizh-info.com/2022/10/02/208574/stonehenge-patrimoine-africain-ben-voyons/',
  2: 'https://www.dreuz.info/2022/10/elections-de-mi-mandat-la-campagne-les-sondages-les-tendances-suivi-quotidien-272063.html',
  3: 'https://www.dreuz.info/2022/10/lukraine-est-en-train-de-gagner-la-guerre-272348.html'},
 'text': {1: 'Stonehenge, patrimoine africain ? Ben voyons…\nCes dernières années, l’Histoire toute entière a subi un véritable assaut de la bien-pensance cosmopolite. Si la manœuvre n’est pas nouvelle, la mémoire ayant souvent été un instrument de légitimation aux mains des différents pouvoirs, on dénote un phénomène nouveau, qui a littéralement explosé avec l’arrivée de la cancel culture : le mensonge pur et dur. Un délire qui repose sur l’ignorance la plus crasse, quand ce n’est sur la mauvaise foi. Les faits historiques sont volontairement déformés, on fait dire à l’Histoire ce que l’on veut, ce dont on nécessite pour étayer sa thèse, aussi farfelue que cela pu

In [87]:
dataloader = DataLoader(ds, batch_size=4,
                        shuffle=True, num_workers=0)

In [69]:
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['url'],
          sample_batched['infox'])

0 ['https://lezarceleurs.blogspot.com/2022/07/leurope-occidentale-na-pas-besoin-de.html', 'https://www.breizh-info.com/2022/04/19/185260/covid-19-une-epidemie-quasiment-terminee-avant-lete-au-niveau-mondial/', 'https://www.francesoir.fr/societe-emploi/sncf-culotte-menstruelle-manque-toilettes', 'https://www.francesoir.fr/culture/jean-jacques-sempe-dessinateur-du-petit-nicolas-est-mort-l-age-de-89-ans'] tensor([1., 1., 0., 0.], dtype=torch.float64)
1 ['https://www.profession-gendarme.com/le-bureau-de-lacademie-de-medecine-expose-sa-complicite-dempoisonnement-en-bande-organisee/', 'https://www.lelibrepenseur.org/demonstration-factuelle-que-les-injections-darn-messager-sont-une-arnaque-qui-na-rien-de-scientifique/', 'https://www.francesoir.fr/culture-medias/diplomes-obligatoires-pour-les-influenceurs-chinois-sur-tiktok', 'https://www.breizh-info.com/2022/07/01/188975/residences-secondaires-le-collectif-dispach-reclame-un-statut-de-resident-en-bretagne-et-un-droit-au-logement-pour-tous-les

In [16]:
import patat.ml.lex_analyser
lex = patat.ml.lex_analyser.LexAnalyser()

In [52]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def tokignore(text):
    result = []
    lemmas = lex.get_lemmas_from_text(text)
    for lemma in lemmas:
        if lemma not in ignore_lemmas:
            result.append(lemma)
    return result

tokenizer = tokignore
#tokenizer = lex.get_lemmas_from_text

def yield_tokens(data_iter):
    for row in data_iter:
        yield tokenizer(row['text'])

In [53]:
corpus_iter = iter(ds)

In [54]:
vocab = build_vocab_from_iterator(yield_tokens(corpus_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [55]:
#vocab = build_vocab_from_iterator(yield_tokens(corpus_iter))

In [56]:
len(vocab)

8340

In [57]:
def text_pipeline(text):
    return vocab(tokenizer(text))

In [58]:
def label_pipeline(x):
    return int(x) - 1

In [60]:
text_pipeline('ET voici encore un autre essai')

[616, 30, 600]

In [61]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iter = corpus_iter
#num_class = len(set([label for (label, text) in train_iter]))
num_class = 1
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [63]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = iter(ds[:350])
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [82]:
import time

def train(dataloader):
    model.train()
    total_acc = 0
    total_count = 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [83]:
train_iter = iter(ds[:350])
test_iter = iter(ds[351:])
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

In [84]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
#train_iter, test_iter = iter(ds)
split_train_
split_valid_

train_iter = iter(ds[:350])
test_iter = iter(ds[351:])
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])


#train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,shuffle=True)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=True)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

TypeError: forward() missing 1 required positional argument: 'offsets'

# Mesure et observation des résultats
---

# Sauvegarde des résultats
---

# Conclusions
---

- ...

# Bricolages
---

In [86]:
values = ['a','b','c']
for i,val in enumerate(values):
    print(i,val)

0 a
1 b
2 c
